##### my-zero-to-gbm-proj-assign

## Optiver Realized Volatility Prediction

This dataset contains stock market data relevant to the practical execution of trades in the financial markets. In particular, it includes order book snapshots and executed trades. With one second resolution, it provides a uniquely fine grained look at the micro-structure of modern financial markets.

This is a code competition where only the first few rows of the test set are available for download. The rows that are visible are intended to illustrate the hidden test set format and folder structure. The remainder will only be available to your notebook when it is submitted. The hidden test set contains data that can be used to construct features to predict roughly 150,000 target values. Loading the entire dataset will take slightly more than 3 GB of memory, by our estimation.

This is also a forecasting competition, where the final private leaderboard will be determined using data gathered after the training period closes, which means that the public and private leaderboards will have zero overlap. During the active training stage of the competition a large fraction of the test data will be filler, intended only to ensure the hidden dataset has approximately the same size as the actual test data. The filler data will be removed entirely during the forecasting phase of the competition and replaced with real

In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
import jovian

#### Data Description

book_[train/test].parquet A parquet file partitioned by stock_id. Provides order book data on the most competitive buy and sell orders entered into the market. The top two levels of the book are shared. The first level of the book will be more competitive in price terms, it will then receive execution priority over the second level.

stock_id - ID code for the stock. Not all stock IDs exist in every time bucket. Parquet coerces this column to the categorical data type when loaded; you may wish to convert it to int8.
time_id - ID code for the time bucket. Time IDs are not necessarily sequential but are consistent across all stocks.
seconds_in_bucket - Number of seconds from the start of the bucket, always starting from 0.
bid_price[1/2] - Normalized prices of the most/second most competitive buy level.
ask_price[1/2] - Normalized prices of the most/second most competitive sell level.
bid_size[1/2] - The number of shares on the most/second most competitive buy level.
ask_size[1/2] - The number of shares on the most/second most competitive sell level.
trade_[train/test].parquet A parquet file partitioned by stock_id. Contains data on trades that actually executed. Usually, in the market, there are more passive buy/sell intention updates (book updates) than actual trades, therefore one may expect this file to be more sparse than the order book.

stock_id - Same as above.
time_id - Same as above.
seconds_in_bucket - Same as above. Note that since trade and book data are taken from the same time window and trade data is more sparse in general, this field is not necessarily starting from 0.
price - The average price of executed transactions happening in one second. Prices have been normalized and the average has been weighted by the number of shares traded in each transaction.
size - The sum number of shares traded.
order_count - The number of unique trade orders taking place.
train.csv The ground truth values for the training set.

stock_id - Same as above, but since this is a csv the column will load as an integer instead of categorical.
time_id - Same as above.
target - The realized volatility computed over the 10 minute window following the feature data under the same stock/time_id. There is no overlap between feature and target data. You can find more info in our tutorial notebook.
test.csv Provides the mapping between the other data files and the submission file. As with other test files, most of the data is only available to your notebook upon submission with just the first few rows available for download.

stock_id - Same as above.
time_id - Same as above.
row_id - Unique identifier for the submission row. There is one row for each existing time ID/stock ID pair. Each time window is not necessarily containing every individual stock.
sample_submission.csv - A sample submission file in the correct format.

row_id - Same as in test.csv.
target - Same definition as in train.csv. The benchmark is using the median target value from train.csv.

In [ ]:
# Execute this to save new versions of the notebook
#jovian.commit(project="my-zero-to-gbm-proj-assign")
jovian.commit(filename="my-zero-to-gbm-proj-assign.ipynb")

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
#train = pd.read_csv('../../../data/optiver-realized-volatility-prediction/train.csv')
train = pd.read_csv('D:\\Edu\\Kaggle\\optiver-realized-volatility-prediction\\train.csv')

In [ ]:
train.info()

## Points to be addressed before ML modeling

1. stock_id - ID code for the stock. <font color='red'> Not all stock IDs exist in every time bucket </font>. Parquet coerces this column to the categorical data type when loaded;  <font color='red'> you may wish to convert it to int8 </font>
2. We have missing “seconds_in_bucket” field?.A: That means there is no related market activities during the last second. For book data you can also assume the top-2 level book shape stays the same as the last available book update within the gap seconds, or, in another word, <font color='red'> you can forward fill the missing data point for all field in book data.</font>
3. I'm trying to make trade data fixed sized. Since missing seconds_in_bucket implies no trade happening within that one-second window, is it technically correct to resample trade data to 600 seconds and fill it with zeros?. Hi, it is correct to assume 0 for order count and size. Some assumptions are required for the price, though. A price of 0 might cause issues.
4. the trade data at T seconds contains a 1-second aggregation of executed orders between [T, T+1 second]
5. One time_id represents a unique 20-minutes trading window which is consistent across all stocks As an example, let’s say time_id = 1 is representing a window between 1900-01-01 12:00:00 and 1900-01-01 12:20:00, then the book data of all stocks for that time_id are is taken from the same window. The data in the first 10 minutes window is shared with all of you, while the order book data of the second 10-minutes is used to build the target for you to predict. The dataset is rolling in such a way that feature and target data will always have zero overlap. Note that time_id is randomly shuffled, so it will not contain any information other than serving as a bridge between different dataset.
6. We can demonstrate the data structure in below way:
<img src = https://www.optiver.com/wp-content/uploads/2021/05/DataBucketing.png>
7. In our competition, we shared the last snapshot of order book for each second. Imagine you have a time_id starting from 1900-01-01 12:00:00, the book update data on seconds_in_bucket = 1 represents the last snapshot of order book update between 12:00:00 and 12:00:01. Similarly to order book data in terms of granularity, but the trade data represents the aggregation of all individual orders happened within one second.
8. So per stock, under the same time_id, the trade data on seconds_in_bucket = 1 represents the aggregation of all individual executed orders between 12:00:00 and 12:00:01. The size is the sum of the size in each individual order, while the price is aggregated as a volume weighted average price of all trades. A straightforward WAP formula can be found on Investopedia.
9. Q: Why we have missing “seconds_in_bucket” field?
A: That means there is no related market activities during the last second.

For book data you can also assume the top-2 level book shape stays the same as the last available book update within the gap seconds, or, in another word, you can forward fill the missing data point for all field in book data. For trade data, it implies no trade happening within that one-second window. One thing to note that trade data tends to be more sparse than book data in many cases.

10. 


Taking the first row of data, it implies that the realized vol of the target bucket for time_id 5, stock_id 0 is 0.004136. How does the book and trade data in feature bucket look like for us to build signals?

In [ ]:
book_example = pd.read_parquet('D:\\Edu\\Kaggle\\optiver-realized-volatility-prediction\\book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('D:\\Edu\\Kaggle\\optiver-realized-volatility-prediction\\trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

#### Book data snapshot

In [ ]:
book_example.info()

### Trade Book Snapshot

In [ ]:
trade_example.info()

##### Realized volatility calculation

our target is to predict short-term realized volatility. Although the order book and trade data for the target cannot be shared, we can still present the realized volatility calculation using the feature data we provided.

As realized volatility is a statistical measure of price changes on a given stock, to calculate the price change we first need to have a stock valuation at the fixed interval (1 second). We will use weighted averaged price, or WAP, of the order book data we provided.

In [ ]:
book_example['bid_size'] = (book_example['bid_size1'] + book_example['bid_size2'])
book_example['bid_price'] = (book_example['bid_price1'] * book_example['bid_size1'] + book_example['bid_price2'] * book_example['bid_size2'])/book_example['bid_size']

book_example['ask_size'] = (book_example['ask_size1'] + book_example['ask_size2'])
book_example['ask_price'] = (book_example['ask_price1'] * book_example['ask_size1'] + book_example['ask_price2'] * book_example['ask_size2'])/(book_example['ask_size1'] + book_example['ask_size2'])

book_example['wap'] = (book_example['bid_price'] * book_example['ask_size'] + book_example['ask_price'] * book_example['bid_size']) / (book_example['bid_size'] +  book_example['ask_size'])

book_example['wap1'] = (book_example['bid_price1'] * book_example['ask_size1'] + book_example['ask_price1'] * book_example['bid_size1']) / (book_example['bid_size1'] +  book_example['ask_size1'])

book_example['wap2'] = (book_example['bid_price2'] * book_example['ask_size2'] + book_example['ask_price2'] * book_example['bid_size2']) / (book_example['bid_size2'] +  book_example['ask_size2'])


The WAP of the stock is plotted below

In [ ]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

### Log returns

How can we compare the price of a stock between yesterday and today?

The easiest method would be to just take the difference. This is definitely the most intuitive way, however price differences are not always comparable across stocks. For example, let's assume that we have invested \$ 1000 in both stock A and stock B and that stock A moves from  \$ 100 to  \$ 102 and stock B moves from  \$ 10 to  \$ 11. We had a total of 10 shares of A ( \$1000 / \$100=10 ) which led to a profit of  10⋅(\$102−\$100)=\$20  and a total of 100 shares of B that yielded \$100. So the price increase was larger for stock A, although the move was proportionally much larger for stock B.

We can solve the above problem by dividing the move by the starting price of the stock, effectively computing the percentage change in price, also known as the stock return. In our example, the return for stock A was  \$102−\$100/\$100=2% , while for stock B it was  \$11−\$10/\$10=10% . The stock return coincides with the percentage change in our invested capital.

Returns are widely used in finance, however log returns are preferred whenever some mathematical modelling is required. Calling  St  the price of the stock  S  at time  t , we can define the log return between  t1  and  t2  as:

 - r(t1,t2)=log(St2/St1)
 
Usually, we look at log returns over fixed time intervals, so with 10-minute log return we mean  rt=rt−10min,t .

Log returns present several advantages, for example:

- they are additive across time  r(t1,t2) + r(t2,t3) = (rt1,t3) 
- regular returns cannot go below -100%, while log returns are not bounded

Next we will compute the log return

To compute the Log Return, we will simply take the logarithm of the ratio between two consecutive WAP. The first row will have an empty return as the previous book update is unknown, therefore the empty return data point will be dropped.

In [ ]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

In [ ]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])


In [ ]:
book_example['log_return'] = book_example['log_return'].fillna(book_example['log_return'].mean())

In [ ]:
book_example.head()

In [ ]:
#~ means NOT 
# also the null condition is applied because when we .diff() method the first row will have empty value as we cannot have diff of return with respect to the previous time id as the first row is the very first time id
book_example = book_example[~book_example['log_return'].isnull()]

In [ ]:
book_example['log_return'].isnull().sum()

Let's plot the tick-to-tick return of this instrument over this time bucket

In [ ]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

###Realized volatility

When we trade options, a valuable input to our models is the standard deviation of the stock log returns. The standard deviation will be different for log returns computed over longer or shorter intervals, for this reason it is usually normalized to a 1-year period and the annualized standard deviation is called volatility.

In this competition, you will be given 10 minutes of book data and we ask you to predict what the volatility will be in the following 10 minutes. Volatility will be measured as follows:

We will compute the log returns over all consecutive book updates and we define the realized volatility,  σ , as the squared root of the sum of squared log returns.

\sigma = \sqrt{\sum_{t}r_{t-1, t}^2}
 
Where we use WAP as price of the stock to compute log returns.

We want to keep definitions as simple and clear as possible, so that Kagglers without financial knowledge will not be penalized. So we are not annualizing the volatility and we are assuming that log returns have 0 mean.


The realized vol of stock 0 in this feature bucket, will be:

In [ ]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

### Naive prediction: using past realized volatility as target

A commonly known fact about volatility is that it tends to be autocorrelated. We can use this property to implement a naive model that just "predicts" realized volatility by using whatever the realized volatility was in the initial 10 minutes.

Let's calculate the past realized volatility across the training set to see how predictive a single naive signal can be.

In [2]:
import glob
list_order_book_file_train = glob.glob('D:\\Edu\\Kaggle\\optiver-realized-volatility-prediction\\book_train.parquet/*')
list_trade_book_file_train = glob.glob('D:\\Edu\\Kaggle\\optiver-realized-volatility-prediction\\trade_train.parquet/*')
#list_order_book_file_train

As the data is partitioned by stock_id (each stock id is one folder), we try to calculcate realized volatility stock by stock and combine them into one target file. Note that the stock id as the partition column is not present if we load the single file so we will remedy that manually. We will reuse the log return and realized volatility functions defined in the previous session.

In [3]:
def compute_wap(df_stock_book):
       df_stock_book['bid_price'] = (df_stock_book['bid_price1'] * df_stock_book['bid_size1'] + df_stock_book['bid_price2'] * df_stock_book['bid_size2'])/(df_stock_book['bid_size1'] + df_stock_book['bid_size2'])
       df_stock_book['bid_size'] = (df_stock_book['bid_size1'] + df_stock_book['bid_size2'])

       df_stock_book['ask_price'] = (df_stock_book['ask_price1'] * df_stock_book['ask_size1'] + df_stock_book['ask_price2'] * df_stock_book['ask_size2'])/(df_stock_book['ask_size1'] + df_stock_book['ask_size2'])
       df_stock_book['ask_size'] = (df_stock_book['ask_size1'] + df_stock_book['ask_size2'])

       df_stock_book['wap'] = (df_stock_book['bid_price'] * df_stock_book['ask_size'] + df_stock_book['ask_price'] * df_stock_book['bid_size']) / (df_stock_book['bid_size'] +  df_stock_book['ask_size'])
       return df_stock_book['wap']

In [ ]:
# def realized_volatility_per_time_id(file_path, prediction_column_name):
#     df_book_data = pd.read_parquet(file_path)
# #    df_book_data['wap'] = (df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
# #                                     df_book_data['bid_size1']+ df_book_data[
# #                                  'ask_size1'])
#     df_book_data['wap'] = compute_wap(df_book_data)
#     df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
#     df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
#     df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
#     df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
#     stock_id = file_path.split('=')[1]
#     df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
#     return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
# df_order_book = pd.DataFrame()
# for file in list_order_book_file_train:
#      df_stock_book = pd.read_parquet(file)
#      df_stock_book['stock_id'] = file.split('=')[1]
#      df_order_book = pd.concat([df_order_book,df_stock_book])


In [4]:
import random
# # Get current state and store
# state = random.getstate()
# # set current state
# random.setstate(state)
random.seed(42)
list_order_book_file_train_50 = random.sample(list_order_book_file_train, 50)
list_trade_book_file_train_50 = [ l.replace('book','trade') for l in list_order_book_file_train_50]
#list_trade_book_file_train_50 = random.sample(list_trade_book_file_train, 50)
# list_order_book_file_train_50_old = list_order_book_file_train_50
# random.seed(42)
# list_order_book_file_train_50 = random.sample(list_order_book_file_train, 5)
# assert(list_order_book_file_train_50 == list_order_book_file_train_50_old)
#list(zip(list_order_book_file_train_50,list_order_book_file_train_50_old))

In [ ]:
!pip install pyarrow

In [ ]:
pd.io.parquet.get_engine('auto')
pd.io.parquet.PyArrowImpl()

In [5]:
# df_order_book_50 = pd.DataFrame()
# df_trade_book_50 = pd.DataFrame()
# for file in list_order_book_file_train_50:
#      df_stock_book = pd.read_parquet(file)
#      df_stock_book['stock_id'] = file.split('=')[1]
#      df_order_book_50 = pd.concat([df_order_book_50,df_stock_book])



df_order_book_5 = pd.DataFrame()
for file in list_order_book_file_train_50[:5]:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_order_book_5 = pd.concat([df_order_book_5,df_stock_book])


df_trade_book_5 = pd.DataFrame()
for file in list_trade_book_file_train_50[:5]:
     df_stock_book = pd.read_parquet(file)
     df_stock_book['stock_id'] = file.split('=')[1]
     df_trade_book_5 = pd.concat([df_trade_book_5,df_stock_book])


In [ ]:
df_order_book_5.stock_id.unique(),df_trade_book_5.stock_id.unique()


In [ ]:
# df_order_book = pd.DataFrame()
# for file in list_order_book_file_train:
#      df_stock_book = pd.read_parquet(file)
#      df_stock_book['stock_id'] = file.split('=')[1]
#      df_order_book = pd.concat([df_order_book,df_stock_book])


# df_trade_book = pd.DataFrame()
# for file in list_trade_book_file_train:
#      df_stock_book = pd.read_parquet(file)
#      df_stock_book['stock_id'] = file.split('=')[1]
#      df_trade_book = pd.concat([df_trade_book,df_stock_book])


In [ ]:
#grouped.filter(a_lt_112)
#len(df_order_book_50.loc[df_order_book_50.seconds_in_bucket == 0])
#print(df_order_book_50.time_id.nunique(),df_order_book_50.stock_id.nunique(),df_order_book_50.seconds_in_bucket.nunique())
#There are a total of 3830 time series, 112 stocks and 600 seconds bucket 
# In this 3830 Times Ids there are a maximum possible order booking of 3830*600 = 2,298,000. But the actual is  

In [6]:
# with open('book_index.csv','w+') as f:
#     for items in list(df_order_book_50.index):
#         f.write('%s\n' %items)
#new_index
df_order_book_5.reset_index(inplace=True,drop=True)
df_trade_book_5.reset_index(inplace=True,drop=True)

# df_order_book.reset_index(inplace=True,drop=True)
# df_trade_book.reset_index(inplace=True,drop=True)

#df_order_book_50.head(-1)
#df_order_book_50[df_order_book_50.index != df_order_book_50['index']]
#(72311913, 11)

In [8]:
#df_order_book_5.index
df_order_book_5_idxed = df_order_book_5.copy()
df_trade_book_5_idxed = df_trade_book_5.copy()


In [ ]:
# df_order_book_50_idxed.drop(columns=['index', 'bid_price1', 'ask_price1',
#        'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2',
#        'ask_size2'],inplace=True)

In [7]:
new_index = pd.Index(np.arange(0,600), name="seconds_in_bucket")

In [ ]:
print(df_order_book_5_idxed.shape, df_trade_book_5_idxed.shape)

In [ ]:
df_order_book_5_idxed.stock_id.unique()

In [ ]:
df_trade_book_5_idxed.stock_id.unique()

In [ ]:
#df_order_book_50_idxed.reindex( pd.MultiIndex.from_product([df_order_book_50_idxed['time_id'],df_order_book_50_idxed['stock_id'],new_index], names=['time_id', 'stock_id','seconds_in_bucket']))
#df_order_book_50_idxed.reindex(pd.MultiIndex.from_frame(df_order_book_50_idxed[['time_id','stock_id','seconds_in_bucket']]))

In [ ]:
#print(df_order_book_50_idxed.time_id.nunique(),df_order_book_50_idxed.stock_id.nunique(),df_order_book_50_idxed.seconds_in_bucket.nunique())
#df_order_book_50_idxed.set_index(['time_id','stock_id'],inplace=True)
#72311913 rows × 9 columns
# 72312513
# 2,202,009,600

In [ ]:
#for a given time_id and stiock_id itself, the number of "seconds_in_bucket" is not the same. THis is the same across
#df_order_book_50_idxed.reindex(new_index,level=2)

In [9]:
df_order_trade_merged = pd.merge(
    df_order_book_5_idxed,
    df_trade_book_5_idxed,
    how="outer",
    on=['time_id','stock_id','seconds_in_bucket'],
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate="m:m"
)

In [ ]:
df_order_trade_merged.head()

In [ ]:
#df_order_trade_merged_5.info()
#book_example.info()
#trade_example.info()
df_order_trade_merged.stock_id.unique()


In [10]:
df_order_trade_merged['stock_id'] = df_order_trade_merged['stock_id'].astype('int8')

In [11]:
grouped = df_order_trade_merged.groupby(['time_id','stock_id'])
#df_order_book_50_idxed.index.levels[2]

In [12]:
#grouped.seconds_in_bucket.max()
#pd.concat([df_order_book_50_idxed],keys=new_index,names=['secs_in_bucket'])
#nw_idx = pd.MultiIndex.append(df_order_book_50_idxed.index,other=new_index)
#72312513
#df_order_book_50_idxed.reset_index(pd.MultiIndex.from_arrays(arrays=[df_order_book_50_idxed.index.levels[0],df_order_book_50_idxed.index.levels[1],new_index]))
#grouped.count()
#191,473
#191,500 (3830 time_id * 50 stocks). 27 rows are missing where some stock_id is not present for all time_ids.
#114,883,800 rows should be there ideally  (191473 * 600). Actual number of rows are 72,311,913. There is a missing number of 42,571,887 rows during due to the missing secods_in_bucket
#df_order_book_50_idxed.shape
#len(df_order_book_50_idxed.index.levels[0]) * len(df_order_book_50_idxed.index.levels[1]) * len(new_index)
#df_order_book_50_idxed.index.get_level_values(1)
#df_order_book_50_idxed.reindex(pd.MultiIndex.from_product([df_order_book_50_idxed.index,new_index],names=['time_id', 'stock_id','secs_in_bucket']),method='nearest')
#s=grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index).reset_index().ffill()).reset_index(drop=True)

#s1=grouped.apply(lambda x : x.set_index('seconds_in_bucket'))
# converts the column "seconds_in_bucket" into an index along with the other 2 index introduced by the GroupBy operation 
# it is as good as groupby (['time_id', 'stock_id','seconds_in_bucket'])
# No use because the missing sequences is not introduced yet

s2=grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index))
# the best of all
# creates a multiindex(['time_id', 'stock_id','seconds_in_bucket']) and also does not reintroduce the column "seconds_in_bucket" as a normal column into the dataframe

#s3=grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index).reset_index())
# creates a multiindex(['time_id', 'stock_id','seconds_in_bucket']) but reintroduces the column "seconds_in_bucket" as a normal column into the dataframe
# 2nd best

#s5=grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index).reset_index(drop=True))
# creates a multiindex(['time_id', 'stock_id',None]). The "seconds_in_bucket" is used for the new sequential ordering but it is removed as index column name and also as removed a normal column. 
# the sequence introduced by the new_index only is used in the multiindex

#s4=grouped.apply(lambda x : x.set_index('seconds_in_bucket').reindex(new_index).reset_index()).reset_index(drop=True)
# resets the indx back to the original rangeIndex but introduces the sequential order. 

#s6=grouped.apply(lambda x : x.reindex(new_index))
#introduce a new index as per names and values of "new_index" and the existing columns ['time_id', 'stock_id','seconds_in_bucket'] stay as it is.
# BADLY resets all the values of all the columns in the dataframe to null (except of coursse the Index column values)

#We can use the drop parameter to avoid the old index being added as a column:



In [13]:
s2.drop(columns=['time_id','stock_id'],inplace=True)

In [56]:
s2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11489400 entries, (5, 2, 0) to (32767, 111, 599)
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bid_price1   float32
 1   ask_price1   float32
 2   bid_price2   float32
 3   ask_price2   float32
 4   bid_size1    float64
 5   ask_size1    float64
 6   bid_size2    float64
 7   ask_size2    float64
 8   price        float32
 9   size         float64
 10  order_count  float64
dtypes: float32(5), float64(6)
memory usage: 799.9 MB


In [57]:
print(s2.shape)

(11489400, 11)


In [58]:
print(s2.index)

MultiIndex([(    5,   2,   0),
            (    5,   2,   1),
            (    5,   2,   2),
            (    5,   2,   3),
            (    5,   2,   4),
            (    5,   2,   5),
            (    5,   2,   6),
            (    5,   2,   7),
            (    5,   2,   8),
            (    5,   2,   9),
            ...
            (32767, 111, 590),
            (32767, 111, 591),
            (32767, 111, 592),
            (32767, 111, 593),
            (32767, 111, 594),
            (32767, 111, 595),
            (32767, 111, 596),
            (32767, 111, 597),
            (32767, 111, 598),
            (32767, 111, 599)],
           names=['time_id', 'stock_id', 'seconds_in_bucket'], length=11489400)


In [59]:
#s2[['bid_size1','ask_size1','bid_size2','ask_size2']] = s2[['bid_size1','ask_size1','bid_size2','ask_size2']].astype('int16')
#s2['stock_id'] = s2['stock_id'].astype('int8')
# s.info()
# #s = s.reindex(columns=['time_id','stock_id','seconds_in_bucket'])

In [60]:
s2.head()

bid_price1  ask_price1  bid_price2  \
time_id stock_id seconds_in_bucket                                       
5       2        0                    1.000607    1.000769    1.000526   
                 1                    1.000526    1.000769    1.000445   
                 2                    1.000526    1.000769    1.000364   
                 3                    1.000607    1.000769    1.000526   
                 4                    1.000526    1.000769    1.000445   

                                    ask_price2  bid_size1  ask_size1  \
time_id stock_id seconds_in_bucket                                     
5       2        0                     1.00085      100.0      200.0   
                 1                     1.00085        1.0      200.0   
                 2                     1.00085        1.0      300.0   
                 3                     1.00085      100.0      300.0   
                 4                     1.00085        1.0      300.0   

                                    bid_size2  ask_size2     price   size  \
time_id stock_id seconds_in_bucket                                          
5       2        0                        1.0        1.0  1.000688  101.0   
                 1                      100.0      101.0       NaN    NaN   
                 2                      112.0      101.0       NaN    NaN   
                 3                        1.0        1.0       NaN    NaN   
                 4                      300.0      101.0       NaN    NaN   

                                    order_count  
time_id stock_id seconds_in_bucket               
5       2        0                          2.0  
                 1                          NaN  
                 2                          NaN  
                 3                          NaN  
                 4                          NaN

In [61]:
s2.isnull().sum()
#3029793

bid_price1     3029793
ask_price1     3029793
bid_price2     3029793
ask_price2     3029793
bid_size1      3029793
ask_size1      3029793
bid_size2      3029793
ask_size2      3029793
price          9393476
size           9393476
order_count    9393476
dtype: int64

In [62]:
# #s.drop(columns=['index'],inplace=True)
# nw_idx = pd.MultiIndex.from_frame(s[['time_id', 'stock_id','seconds_in_bucket']])
# nw_idx.set_axis(nw_idx,axis = 1, inplace = False)
# s.reindex(index=nw_idx, columns = ['bid_price1','ask_price1','bid_price2','ask_price2','bid_size1','ask_size1','bid_size2','ask_size2'],copy=False)
#s.set_index(['time_id', 'stock_id','seconds_in_bucket'],inplace=True)

In [14]:
s2[['size','order_count']] = s2[['size','order_count']].fillna(0) 


In [15]:
s2.isna().sum()

bid_price1     3029793
ask_price1     3029793
bid_price2     3029793
ask_price2     3029793
bid_size1      3029793
ask_size1      3029793
bid_size2      3029793
ask_size2      3029793
price          9393476
size                 0
order_count          0
dtype: int64

In [69]:
s2.loc(axis=0)[:,:,0].isna().sum()


bid_price1         0
ask_price1         0
bid_price2         0
ask_price2         0
bid_size1          0
ask_size1          0
bid_size2          0
ask_size2          0
price          14413
size               0
order_count        0
dtype: int64

In [70]:
s2.loc(axis=0)[:,:,0][s2.loc(axis=0)[:,:,0]['price'].isna()]

bid_price1  ask_price1  bid_price2  \
time_id stock_id seconds_in_bucket                                       
5       68       0                    1.000354    1.000472    1.000236   
        100      0                    0.999077    1.000103    0.999009   
        111      0                    1.001884    1.002048    1.001720   
11      68       0                    0.999941    1.000059    0.999824   
        81       0                    0.996471    0.996785    0.996419   
...                                        ...         ...         ...   
32763   68       0                    0.998702    0.998975    0.998565   
        100      0                    1.004087    1.004697    1.003721   
32767   2        0                    0.999298    0.999454    0.999220   
        68       0                    1.000636    1.000764    1.000509   
        81       0                    0.998934    0.999645    0.998815   

                                    ask_price2  bid_size1  ask_size1  \
time_id stock_id seconds_in_bucket                                     
5       68       0                    1.000591      101.0       40.0   
        100      0                    1.000239      100.0        7.0   
        111      0                    1.002211      500.0      509.0   
11      68       0                    1.000176      100.0      200.0   
        81       0                    0.997569      286.0      201.0   
...                                        ...        ...        ...   
32763   68       0                    0.999112      123.0      201.0   
        100      0                    1.004941      100.0        9.0   
32767   2        0                    0.999532      408.0      300.0   
        68       0                    1.000891      300.0      211.0   
        81       0                    0.999763      100.0      200.0   

                                    bid_size2  ask_size2  price  size  \
time_id stock_id seconds_in_bucket                                      
5       68       0                      201.0      321.0    NaN   0.0   
        100      0                       64.0      100.0    NaN   0.0   
        111      0                      710.0     2091.0    NaN   0.0   
11      68       0                      200.0      300.0    NaN   0.0   
        81       0                       20.0      100.0    NaN   0.0   
...                                       ...        ...    ...   ...   
32763   68       0                      396.0      100.0    NaN   0.0   
        100      0                      200.0      200.0    NaN   0.0   
32767   2        0                      600.0      500.0    NaN   0.0   
        68       0                      400.0        8.0    NaN   0.0   
        81       0                      300.0      100.0    NaN   0.0   

                                    order_count  
time_id stock_id seconds_in_bucket               
5       68       0                          0.0  
        100      0                          0.0  
        111      0                          0.0  
11      68       0                          0.0  
        81       0                          0.0  
...                                         ...  
32763   68       0                          0.0  
        100      0                          0.0  
32767   2        0                          0.0  
        68       0                          0.0  
        81       0                          0.0  

[14413 rows x 11 columns]

In [ ]:
s2.head(10)

In [67]:
s2.loc[32767,2,:].head(10)

bid_price1  ask_price1  bid_price2  \
time_id stock_id seconds_in_bucket                                       
32767   2        0                    0.999298    0.999454    0.999220   
                 1                    0.999298    0.999454    0.999220   
                 2                         NaN         NaN         NaN   
                 3                    0.999298    0.999376    0.999220   
                 4                    0.999298    0.999376    0.999220   
                 5                    0.999298    0.999376    0.999220   
                 6                    0.999220    0.999298    0.999143   
                 7                    0.999220    0.999298    0.999143   
                 8                    0.999220    0.999298    0.999143   
                 9                    0.999220    0.999298    0.999143   

                                    ask_price2  bid_size1  ask_size1  \
time_id stock_id seconds_in_bucket                                     
32767   2        0                    0.999532      408.0      300.0   
                 1                    0.999532      408.0      400.0   
                 2                         NaN        NaN        NaN   
                 3                    0.999454      108.0      300.0   
                 4                    0.999454      308.0      200.0   
                 5                    0.999454      208.0      200.0   
                 6                    0.999376      118.0      200.0   
                 7                    0.999376      108.0      300.0   
                 8                    0.999376      108.0      200.0   
                 9                    0.999376      208.0      200.0   

                                    bid_size2  ask_size2     price   size  \
time_id stock_id seconds_in_bucket                                          
32767   2        0                      600.0      500.0       NaN    0.0   
                 1                      600.0      500.0       NaN    0.0   
                 2                        NaN        NaN       NaN    0.0   
                 3                      600.0      600.0       NaN    0.0   
                 4                      600.0      600.0  0.999298   99.0   
                 5                      600.0      800.0       NaN    0.0   
                 6                      400.0      400.0  0.999298  308.0   
                 7                      400.0      600.0       NaN    0.0   
                 8                      400.0      700.0       NaN    0.0   
                 9                      400.0      700.0       NaN    0.0   

                                    order_count  
time_id stock_id seconds_in_bucket               
32767   2        0                          0.0  
                 1                          0.0  
                 2                          0.0  
                 3                          0.0  
                 4                          1.0  
                 5                          0.0  
                 6                          6.0  
                 7                          0.0  
                 8                          0.0  
                 9                          0.0

In [ ]:
s2.loc[32767,68,:].head(10)

In [ ]:
s2.loc[32767,81,:].head(40)

In [ ]:
s2.loc[32767,100,:].head(1)

In [ ]:
s2.loc[32767,111,:].head(1)

In [ ]:
s2.index

In [ ]:
# s2.fillna(method='bfill',limit=1,inplace=True,axis=0)

In [16]:
bf = s2.groupby(['time_id','stock_id'])

In [17]:
bf['price'].first()

time_id  stock_id
5        2           1.000688
         68          1.000585
         81          1.003518
         100         1.000103
         111         1.002161
                       ...   
32767    2           0.999298
         68          1.000764
         81          0.999430
         100         0.999315
         111         1.000101
Name: price, Length: 19149, dtype: float32

In [18]:
#s2.loc[(:,:,0),'price'] = bf['price'].first()
s2.loc[s2.index.get_level_values(2)==0,'price'] = bf['price'].first()

In [ ]:
s2.loc[5,111,:].head(10)

In [19]:
s2.isna().sum()

bid_price1     3029793
ask_price1     3029793
bid_price2     3029793
ask_price2     3029793
bid_size1      3029793
ask_size1      3029793
bid_size2      3029793
ask_size2      3029793
price          9379063
size                 0
order_count          0
dtype: int64

In [20]:
#s2.loc[:,:,1:] = s2.loc[:,:,1:].fillna(method='ffill')
#s2.loc[:,:,1:].fillna(method='ffill',inplace=True)
s2.fillna(method='ffill',inplace=True)

In [21]:
s2.isna().sum()

bid_price1     0
ask_price1     0
bid_price2     0
ask_price2     0
bid_size1      0
ask_size1      0
bid_size2      0
ask_size2      0
price          0
size           0
order_count    0
dtype: int64

Looping through each individual stocks, we can get the past realized volatility as prediction for each individual stocks.

In [22]:
s2['order_count'] = s2['order_count'].astype('int16')
s2[['bid_size1','ask_size1','bid_size2','ask_size2','size']] = s2[['bid_size1','ask_size1','bid_size2','ask_size2','size']].astype('int32')


In [79]:
s2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11489400 entries, (5, 2, 0) to (32767, 111, 599)
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   bid_price1   float32
 1   ask_price1   float32
 2   bid_price2   float32
 3   ask_price2   float32
 4   bid_size1    int32  
 5   ask_size1    int32  
 6   bid_size2    int32  
 7   ask_size2    int32  
 8   price        float32
 9   size         int32  
 10  order_count  int16  
dtypes: float32(5), int16(1), int32(5)
memory usage: 515.2 MB


In [23]:
# s2['bid_size'] = (s2['bid_size1'] + s2['bid_size2'])
# s2['bid_price'] = (s2['bid_price1'] * s2['bid_size1'] + s2['bid_price2'] * s2['bid_size2'])/(s2['bid_size1'] + s2['bid_size2'])

# s2['ask_size'] = (s2['ask_size1'] + s2['ask_size2'])
# s2['ask_price'] = (s2['ask_price1'] * s2['ask_size1'] + s2['ask_price2'] * s2['ask_size2'])/(s2['ask_size1'] + s2['ask_size2'])

#s2['wap'] = (((s2['bid_price1'] * s2['bid_size1'] + s2['bid_price2'] * s2['bid_size2'])/(s2['bid_size1'] + s2['bid_size2']) * (s2['ask_size1'] + s2['ask_size2'])) + ((s2['ask_price1'] * s2['ask_size1'] + s2['ask_price2'] * s2['ask_size2'])/(s2['ask_size1'] + s2['ask_size2'])) * (s2['bid_size1'] + s2['bid_size2'])) / ((s2['bid_size1'] + s2['bid_size2']) +  (s2['ask_size1'] + s2['ask_size2']))

s2['order_wap'] = (((s2['bid_price1'] * s2['bid_size1'] + s2['bid_price2'] * s2['bid_size2'])/(s2['bid_size1'] + s2['bid_size2']) * (s2['ask_size1'] + s2['ask_size2'])) + ((s2['ask_price1'] * s2['ask_size1'] + s2['ask_price2'] * s2['ask_size2'])/(s2['ask_size1'] + s2['ask_size2'])) * (s2['bid_size1'] + s2['bid_size2'])) / ((s2['bid_size1'] + s2['bid_size2']) +  (s2['ask_size1'] + s2['ask_size2']))


In [24]:
s2['trade_wap'] = s2.groupby(['time_id','stock_id'])['price'].apply(lambda x: x - x.mean())
#s2['trade_wap_sq'] = s2.groupby(['time_id','stock_id'])['price'].apply(lambda x: x - x.mean())

In [25]:
pd.options.display.float_format = '{:,.16f}'.format

In [26]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()

def trade_volatility(trade_wap_prices):
    return np.sqrt(np.sum(trade_wap_prices**2)/len(trade_wap_prices))

def order_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [27]:
s2['log_return'] = s2.groupby(['time_id','stock_id'])['order_wap'].apply(log_return)

In [28]:
s2['log_return'].isna().sum()

19149

In [29]:
#remove the rows with null values of log return
#s2[~s2['log_return'].isnull()]['log_return']
df_realized_vol_per_stock =  pd.DataFrame(s2[~s2['log_return'].isnull()]['log_return'].groupby(['time_id','stock_id']).agg(order_volatility))

In [30]:
df_realized_vol_per_stock.isna().sum()

log_return    0
dtype: int64

In [ ]:
# find the volatility values without converting or removing the null values   
df_realized_vol_per_stock1 =  pd.DataFrame(s2['log_return'].groupby(['time_id','stock_id']).agg(order_volatility))

In [ ]:
assert df_realized_vol_per_stock1.values == df_realized_vol_per_stock.values

In [ ]:

np.testing.assert_array_equal(df_realized_vol_per_stock1.values, df_realized_vol_per_stock.values, err_msg='not matching', verbose=True)

Max absolute difference: 3.7252903e-09
Max relative difference: 1.6754667e-07

This error is ignorable. So we will leave the null values as it is in the log return

In [ ]:
#converting all null values to zeros
df_realized_vol_per_stock2 =  pd.DataFrame(s2['log_return'].fillna(0).groupby(['time_id','stock_id']).agg(order_volatility))

In [ ]:
np.testing.assert_array_equal(df_realized_vol_per_stock1,df_realized_vol_per_stock2) 

In [ ]:
np.testing.assert_array_equal(df_realized_vol_per_stock,df_realized_vol_per_stock2) 

In [31]:
df_realized_vol_per_stock

log_return
time_id stock_id                   
5       2        0.0018733931133985
        68       0.0023004176145590
        81       0.0054617860000476
        100      0.0046436459937157
        111      0.0029989438960921
...                             ...
32767   2        0.0010246982159702
        68       0.0013834203207654
        81       0.0026151711543734
        100      0.0019775423480388
        111      0.0012556054605663

[19149 rows x 1 columns]

In [ ]:
df_realized_vol_per_stock.index.get_level_values(0)

In [32]:
#df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: fff(x.index.get_level_values(1),x.index.get_level_values(0)), axis=1 )
#df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1)+x.index.get_level_values(0))
df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock.apply(lambda x: x.index.get_level_values(1).astype(str) + "-" + x.index.get_level_values(0).astype(str))

#f'{stock_id}-{x}'

In [33]:
df_realized_vol_per_stock['trade_return'] = pd.DataFrame(s2['trade_wap'].groupby(['time_id','stock_id']).agg(trade_volatility))


In [34]:
df_realized_vol_per_stock

log_return     row_id       trade_return
time_id stock_id                                                 
5       2        0.0018733931133985        2-5 0.0005385612603277
        68       0.0023004176145590       68-5 0.0005042065749876
        81       0.0054617860000476       81-5 0.0012333415215835
        100      0.0046436459937157      100-5 0.0010931357974187
        111      0.0029989438960921      111-5 0.0005883899284527
...                             ...        ...                ...
32767   2        0.0010246982159702    2-32767 0.0003467606729828
        68       0.0013834203207654   68-32767 0.0002697620948311
        81       0.0026151711543734   81-32767 0.0005826270207763
        100      0.0019775423480388  100-32767 0.0007258245022967
        111      0.0012556054605663  111-32767 0.0003218931669835

[19149 rows x 3 columns]

In [94]:
# def past_realized_volatility_per_stock(list_file,prediction_column_name):
#     df_past_realized = pd.DataFrame()
#     for file in list_file:
#         df_past_realized = pd.concat([df_past_realized,
#                                      realized_volatility_per_time_id(file,prediction_column_name)])
#     return df_past_realized
# df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
#                                                            prediction_column_name='pred')

Let's join the output dataframe with train.csv to see the performance of the naive prediction on training set.

In [35]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]


We will evaluate the naive prediction result by two metrics: RMSPE and R squared.

In [36]:
df_joined = train.merge(df_realized_vol_per_stock, on = ['row_id'], how = 'inner')

In [37]:
df_joined

,row_id,target,log_return,trade_return
0,2-5,0.0018475460000000,0.0018733931133985,0.0005385612603277
1,2-11,0.0008059180000000,0.0008151048469293,0.0003309202729724
2,2-16,0.0015811100000000,0.0012939132651704,0.0002343668165850
3,2-31,0.0015986860000000,0.0015225304382781,0.0005046058795415
4,2-62,0.0015032210000000,0.0012425119932203,0.0002491848717909
...,...,...,...,...
19144,111-32751,0.0019392000000000,0.0024886340803033,0.0005013425834477
19145,111-32753,0.0008449350000000,0.0009824073836777,0.0002220391470473
19146,111-32758,0.0014585060000000,0.0019381003855530,0.0010461938800290
19147,111-32763,0.0032180480000000,0.0025020021033653,0.0006868820055388


In [38]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

In [ ]:
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['log_return']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['log_return']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')

In [ ]:
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['log_return'] + np.sqrt(df_joined['trade_return'])),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['log_return'] + np.sqrt(df_joined['trade_return'])),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}%')

In [ ]:
jovian.commit(filename="my-zero-to-gbm-proj-assign.ipynb")

## Machine Learning

Now we will start applying ML techniques to predict the volataility of the next 10 minutes window for each time-id/stock-id based on the order book volatility and trade volatility

we will learn the hyper parameters givne using the training targets for the same

we will use 2 different models to do the same

## Graident Bossting

We're now ready to train our gradient boosting machine (GBM) model. Here's how a GBM model works:

In [ ]:
from xgboost import XGBRegressor

## Intel Extension for Scikit-learn

Intel(R) Extension for Scikit-learn* dynamically patches scikit-learn estimators to use Intel(R) oneAPI Data Analytics Library as the underlying solver, while getting the same solution faster.

To install these Intel-optimized packages for scikit-learn on Windows, Mac, and Linux x86_64, simply:

conda install scikit-learn-intelex

Once installed, there are two ways in which you can enable the replacement patching functionality for scikit-learn. You can enable it when you run your application:

python -m sklearnex my_application.py

Or you can explicitly enable the patching in your code:

from sklearnex import patch_sklearn

patch_sklearn()

In [39]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [ ]:
Lazy

### Cross Validation

 create a validation set before training our XGBoost model. We'll use a different validation strategy this time, called <b> ShuffleSplit </b> cross validation (source):

In [40]:
from sklearn.model_selection import ShuffleSplit

In [41]:
ss = ShuffleSplit(n_splits = 5, test_size = 0.25, random_state=42)

Let's define a helper function train_and_evaluate which trains a model the given parameters and returns the trained model, training error and validation error.

In [42]:
# def train_and_evaluate(X_train, train_targets, X_val, val_targets, **params):
#     model = XGBRegressor(random_state=42, n_jobs=-1, **params)
#     model.fit(X_train, train_targets)
#     train_R2 = round(r2_score(train_targets, model.predict(X_train)),3)
#     train_RMSPE = round(rmspe(train_targets, model.predict(X_train)),3)
#     val_R2 = round(r2_score(val_targets, model.predict(X_val)),3)
#     val_RMSPE = round(rmspe(val_targets, model.predict(X_val),),3)

#     # train_rmse = rmse(model.predict(X_train), train_targets)
#     # val_rmse = rmse(model.predict(X_val), val_targets)
#     return model, train_R2, train_RMSPE, val_R2, val_RMSPE
# #n_jobs = -1 means that use all the available threads in that machine where the alogorithm is running 

Now we will train the model for each split data of the ShuffleSplit

In [43]:
df_joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19149 entries, 0 to 19148
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   row_id        19149 non-null  object 
 1   target        19149 non-null  float64
 2   log_return    19149 non-null  float64
 3   trade_return  19149 non-null  float32
dtypes: float32(1), float64(2), object(1)
memory usage: 673.2+ KB


In [44]:
inputs = df_joined[['log_return', 'trade_return']].copy()
targets = df_joined['target'].copy()

In [45]:
inputs
#targets

,log_return,trade_return
0,0.0018733931133985,0.0005385612603277
1,0.0008151048469293,0.0003309202729724
2,0.0012939132651704,0.0002343668165850
3,0.0015225304382781,0.0005046058795415
4,0.0012425119932203,0.0002491848717909
...,...,...
19144,0.0024886340803033,0.0005013425834477
19145,0.0009824073836777,0.0002220391470473
19146,0.0019381003855530,0.0010461938800290
19147,0.0025020021033653,0.0006868820055388


In [46]:
from lazypredict.Supervised import LazyRegressor


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [47]:
# models = []

# for train_idxs, val_idxs in ss.split(inputs):
#     X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#     X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#     model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                      train_targets, 
#                                                      X_val, 
#                                                      val_targets, 
#                                                      max_depth=5, 
#                                                      n_estimators=50)
#     models.append(model)
#     print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))

Let's also define a function to average predictions from the 5 different models.

In [ ]:
def predict_avg(models, inputs):
    return np.mean([model.predict(inputs) for model in models], axis=0)

In [ ]:
preds = predict_avg(models, inputs)

In [ ]:
preds

In [48]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV

ImportError: cannot import name 'enable_halving_search_cv' from 'sklearn.experimental' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\__init__.py)

In [49]:
# model = XGBRegressor(random_state=42, n_jobs=-1)
# param_grid = {"max_depth": [[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, None]],
#               "n_estimators" : list(range(10,51,2)),
#               "learning_rate": np.logspace(-3,3,100),
#               # "booster": ['gbtree', 'gblinear','dart'],
#               # "gamma": np.logspace(-1,1,100),
#               "subsample": np.linspace(0.1,1.0,9, endpoint=False),
#               # "num_parallel_tree": range(1,6,1),
#               # "reg_alpha": np.logspace(-3,3,300),
#               # "reg_lambda": np.logspace(-3,3,300)
#             }


In [50]:
# models = []
# for train_idxs, val_idxs in ss.split(inputs):
#     X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
#     X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
#     grid_search = HalvingGridSearchCV(model, param_grid, random_state=42).fit(X_train, train_targets)
#     model, train_R2, train_RMSPE, val_R2, val_RMSPE = train_and_evaluate(X_train, 
#                                                     train_targets, 
#                                                     X_val, 
#                                                     val_targets, 
#                                                     **grid_search.best_params_)
#     models.append(model)
#     print('Train R2: {}, Train RMSPE: {}, Validation R2: {}, Validation RMSPE: {}'.format(train_R2, train_RMSPE, val_R2, val_RMSPE))


In [51]:
models = []
for train_idxs, val_idxs in ss.split(inputs):
    X_train, train_targets = inputs.iloc[train_idxs], targets.iloc[train_idxs]
    X_val, val_targets = inputs.iloc[val_idxs], targets.iloc[val_idxs]
    X_train = np.ascontiguousarray(X_train).reshape(-1,2)
    train_targets = np.ascontiguousarray(train_targets).reshape(-1,1)
    X_val = np.ascontiguousarray(X_val).reshape(-1,2)
    val_targets = np.ascontiguousarray(val_targets).reshape(-1,1)
    reg = LazyRegressor(ignore_warnings=False, random_state=11111, verbose=True)
    models, predictions = reg.fit(X_train, X_val, train_targets, val_targets)  # pass all sets
    print(models.head(100))


  2%|▏         | 1/50 [00:01<00:49,  1.01s/it]

{'Model': 'AdaBoostRegressor', 'R-Squared': 0.20996987801794598, 'Adjusted R-Squared': 0.2096396668906808, 'RMSE': 0.002632727723268154, 'Time taken': 1.011321783065796}


  4%|▍         | 2/50 [00:01<00:33,  1.43it/s]

{'Model': 'BaggingRegressor', 'R-Squared': 0.701250799510781, 'Adjusted R-Squared': 0.7011259304614647, 'RMSE': 0.0016189651396941558, 'Time taken': 0.4757659435272217}


  8%|▊         | 4/50 [00:02<00:19,  2.39it/s]

...

SVR model failed to execute
Input model support vectors are empty
{'Model': 'SVR', 'R-Squared': -28.14681106602343, 'Adjusted R-Squared': -28.158993641181645, 'RMSE': 0.016309501987898583, 'Time taken': 0.019946813583374023}
{'Model': 'TransformedTargetRegressor', 'R-Squared': 0.7243080098032972, 'Adjusted R-Squared': 0.7241927780414595, 'RMSE': 0.0015861974646875804, 'Time taken': 0.028922557830810547}
{'Model': 'TweedieRegressor', 'R-Squared': 0.5849868415444545, 'Adjusted R-Squared': 0.5848133773193946, 'RMSE': 0.0019461491033078177, 'Time taken': 0.018948078155517578}


 98%|█████████▊| 49/50 [03:29<00:01,  1.74s/it]

{'Model': 'XGBRegressor', 'R-Squared': 0.7379254588192357, 'Adjusted R-Squared': 0.7378159187811246, 'RMSE': 0.0015465273121664687, 'Time taken': 0.732041597366333}


100%|██████████| 50/50 [03:29<00:00,  4.20s/it]

{'Model': 'LGBMRegressor', 'R-Squared': 0.7497008893093954, 'Adjusted R-Squared': 0.7495962710813115, 'RMSE': 0.0015113840350644957, 'Time taken': 0.22539854049682617}
                               Adjusted R-Squared  R-Squared  RMSE  Time Taken
Model                                                                         
LGBMRegressor                                0.75       0.75  0.00        0.23
HistGradientBoostingRegressor                0.75       0.75  0.00        0.45
GradientBoostingRegressor                    0.75       0.75  0.00        1.39
XGBRegressor                                 0.74       0.74  0.00        0.73
KNeighborsRegressor                          0.73       0.73  0.00        0.10
KNeighborsRegressor                          0.73       0.73  0.00        0.20
SGDRegressor                                 0.73       0.73  0.00        0.04
LinearRegression                             0.72       0.72  0.00        0.03
LinearRegression                          

In [ ]:
# Fit LazyRegressor




In [ ]:
grid_search.best_params_

Discussion on cross validation designs

https://www.kaggle.com/vishnurapps/undersanding-kfold-stratifiedkfold-and-groupkfold